In [55]:
%pip install sentence-transformers

In [56]:
from sentence_transformers import SentenceTransformer,util

In [57]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [58]:
question_pool = [
    "Hi",
    "What's on the menu?",
    "Do you have burgers?",
    "Tell me about pizzas",
    "What types of pasta do you serve?",
    "Do you offer biryani?",
    "Do you have BBQ items?",
    "Can I get a dessert?",
    "What drinks do you serve?",
    "Can I book a table?",
    "Thanks!",
    "Bye"
]

# Define the corresponding fun answer pool
answer_pool = [
    "Hey there! Welcome to Foodie's Paradise! What would you like today?",
    "Our menu is packed with love! 🍕🍔🍝 We have pizzas, burgers, pasta, biryani, BBQ, desserts, and much more!",
    "Our burgers are juicy and delicious! 🍔 Options: Classic Beef, Chicken Supreme, Veggie Delight.",
    "Hot and cheesy pizzas await you! 🍕 Options: Margherita, Pepperoni, BBQ Chicken, Veggie Special.",
    "We serve creamy Alfredo, spicy Arrabiata, and classic Bolognese pasta! 🍝",
    "Aromatic biryani for you! Options: Chicken Biryani, Mutton Biryani, and Veg Biryani. 🍛",
    "Smoky and tender BBQ dishes! Options: BBQ Wings, Ribs, and BBQ Platters. 🔥",
    "Dessert time! 🍰 Options: Chocolate Lava Cake, Cheesecake, Ice Cream Sundae.",
    "Refreshing drinks available! 🥤 Options: Lemonade, Mojito, Cold Coffee, Fresh Juices.",
    "Sure! 📅 To book a table, please call us at +123-456-7890 or visit our website to reserve online!",
    "You're welcome! 😄 Enjoy your meal!",
    "Goodbye foodie! 👋 Come back hungry!"
]



In [59]:
question_embeddings = model.encode(question_pool, convert_to_tensor=True)

In [60]:
import numpy as np

In [61]:
question_embeddings = np.array(question_embeddings).astype('float32')

In [62]:
np.save('restaurant_embeddings.npy',question_embeddings)

In [63]:
question_embeddings = np.load('restaurant_embeddings.npy')

In [64]:
%pip install faiss-cpu

In [65]:
import faiss

In [66]:
dimension = question_embeddings.shape[1]  # usually 384
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)

In [67]:
SIMILARITY_THRESHOLD = 0.4

In [68]:
def chatbot_reply(user_input):
    user_embedding = model.encode(user_input, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(user_embedding, question_embeddings)

    best_score = cosine_scores.max().item()
    best_index = cosine_scores.argmax().item()

    if best_score >= SIMILARITY_THRESHOLD:
        return answer_pool[best_index]
    else:
        return "I'm not sure about that. Try asking about food, drinks, or bookings!"

# Chatbot interaction loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Bot: Goodbye! 👋")
        break
    response = chatbot_reply(user_input)
    print("Bot:", response)

You: hello
Bot: Hey there! Welcome to Foodie's Paradise! What would you like today?
You: tell me about burgers
Bot: Our burgers are juicy and delicious! 🍔 Options: Classic Beef, Chicken Supreme, Veggie Delight.
You: do you have drinks
Bot: Refreshing drinks available! 🥤 Options: Lemonade, Mojito, Cold Coffee, Fresh Juices.
You: can i reserve a seat
Bot: Sure! 📅 To book a table, please call us at +123-456-7890 or visit our website to reserve online!
You: thank you
Bot: You're welcome! 😄 Enjoy your meal!
You: bye
Bot: Goodbye! 👋
